In [1]:
import pandas as pd
import numpy as np

# not skipping empty values, to demonstrate data preprocessing steps later
df = pd.read_csv('datasets/online_retail.csv', na_filter=False, encoding='cp1252')

df.info()

C:\Users\Computer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532619 entries, 0 to 532618
Data columns (total 8 columns):
InvoiceNo      532619 non-null object
StockCode      532619 non-null object
Description    532619 non-null object
Quantity       532619 non-null int64
InvoiceDate    532619 non-null object
UnitPrice      532619 non-null float64
CustomerID     532619 non-null object
Country        532619 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 32.5+ MB


In [2]:
#the variable Description represents the same thing as StockCode, so we drop it.
df.drop(['Description'],axis=1,inplace=True)
print(df.head(10))

  InvoiceNo StockCode  Quantity     InvoiceDate  UnitPrice CustomerID  \
0    536365    85123A         6  1/12/2010 8:26       2.55      17850   
1    536365     71053         6  1/12/2010 8:26       3.39      17850   
2    536365    84406B         8  1/12/2010 8:26       2.75      17850   
3    536365    84029G         6  1/12/2010 8:26       3.39      17850   
4    536365    84029E         6  1/12/2010 8:26       3.39      17850   
5    536365     22752         2  1/12/2010 8:26       7.65      17850   
6    536365     21730         6  1/12/2010 8:26       4.25      17850   
7    536366     22633         6  1/12/2010 8:28       1.85      17850   
8    536366     22632         6  1/12/2010 8:28       1.85      17850   
9    536367     84879        32  1/12/2010 8:34       1.69      13047   

          Country  
0  United Kingdom  
1  United Kingdom  
2  United Kingdom  
3  United Kingdom  
4  United Kingdom  
5  United Kingdom  
6  United Kingdom  
7  United Kingdom  
8  United Kingdo

In [3]:
df = df.replace({'StockCode': r'[A-Z]'}, {'StockCode': ''}, regex=True)

In [4]:
# group by transactions, then list all products
transactions = df.groupby(['CustomerID'])['StockCode'].apply(list)



In [5]:
print(transactions.head(5))

CustomerID
         [22139, 21773, 21774, 21786, 21787, 21790, 217...
12346                                              [23166]
12347    [85116, 22375, 71477, 22492, 22771, 22772, 227...
12348    [84992, 22951, 84991, 84991, 21213, 21213, 226...
12349    [23112, 23460, 21564, 21411, 21563, 22131, 221...
Name: StockCode, dtype: object


In [6]:
from apyori import apriori

# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.02))

# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({''}), support=0.11935483870967742, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({''}), confidence=0.11935483870967742, lift=1.0)]), RelationRecord(items=frozenset({'10133'}), support=0.023502304147465437, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10133'}), confidence=0.023502304147465437, lift=1.0)]), RelationRecord(items=frozenset({'10135'}), support=0.02142857142857143, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10135'}), confidence=0.02142857142857143, lift=1.0)]), RelationRecord(items=frozenset({'15036'}), support=0.044930875576036866, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'15036'}), confidence=0.044930875576036866, lift=1.0)]), RelationRecord(items=frozenset({'15044'}), support=0.020046082949308756, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'15044'}),

In [7]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

print(result_df.head(20))

   Left_side Right_side   Support  Confidence  Lift
0                        0.119355    0.119355   1.0
1                 10133  0.023502    0.023502   1.0
2                 10135  0.021429    0.021429   1.0
3                 15036  0.044931    0.044931   1.0
4                 15044  0.020046    0.020046   1.0
5                 15056  0.056912    0.056912   1.0
6                 16156  0.033871    0.033871   1.0
7                 16161  0.065207    0.065207   1.0
8                 16169  0.038940    0.038940   1.0
9                 16237  0.025346    0.025346   1.0
10                17003  0.031567    0.031567   1.0
11                20668  0.025806    0.025806   1.0
12                20674  0.023502    0.023502   1.0
13                20675  0.034562    0.034562   1.0
14                20676  0.042857    0.042857   1.0
15                20677  0.030876    0.030876   1.0
16                20679  0.032719    0.032719   1.0
17                20682  0.032028    0.032028   1.0
18          

In [8]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(10))

                           Left_side Right_side   Support  Confidence  \
15125                    21989,21987      21988  0.020046    0.956044   
15126                    21988,21989      21987  0.020046    0.966667   
4227                           21987      21988  0.022581    0.915888   
4228                           21988      21987  0.022581    0.960784   
9058                           23291      23289  0.021889    0.904762   
9057                           23289      23291  0.021889    0.848214   
39520        22916,22917,22918,22919      22921  0.023963    0.920354   
43766  22917,22916,22918,22920,22919      22921  0.022350    0.915094   
39525        22916,22917,22918,22920      22921  0.023502    0.910714   
29704              22916,22917,22918      22921  0.025115    0.908333   

            Lift  
15125  40.678733  
15126  39.208723  
4227   38.970130  
4228   38.970130  
9058   35.059524  
9057   35.059524  
39520  34.139626  
43766  33.944525  
39525  33.782051  
29704 

In [9]:
result_df = result_df.sort_values(by='Confidence', ascending=False)
print(result_df.head(10))

                           Left_side Right_side   Support  Confidence  \
43651  23174,22698,23173,23171,22699      22697  0.021429         1.0   
43410  22698,23175,23170,22423,22699      22697  0.020276         1.0   
22104              20725,23199,22411      85099  0.020046         1.0   
38605        23202,22386,23199,23203      85099  0.022811         1.0   
38625        23202,23201,22386,23203      85099  0.021429         1.0   
26232              23202,21929,22386      85099  0.021429         1.0   
34475        22386,20725,23199,23203      85099  0.020737         1.0   
26120              23202,22386,21928      85099  0.021198         1.0   
34635        23207,23203,20725,23199      85099  0.020046         1.0   
19420              20712,20725,22384      85099  0.021889         1.0   

            Lift  
43651  11.185567  
43410  11.185567  
22104   5.636364  
38605   5.636364  
38625   5.636364  
26232   5.636364  
34475   5.636364  
26120   5.636364  
34635   5.636364  
19420 

In [10]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44541 entries, 43651 to 891
Data columns (total 5 columns):
Left_side     44541 non-null object
Right_side    44541 non-null object
Support       44541 non-null float64
Confidence    44541 non-null float64
Lift          44541 non-null float64
dtypes: float64(3), object(2)
memory usage: 2.0+ MB


In [11]:
#the productIF of 'HERB MARKER CHIVES' is 22921, obtained from Excel searching
interested = result_df[result_df['Right_side'] == '22921'].append(result_df[result_df['Left_side'] == '22921'])
interested

,Left_side,Right_side,Support,Confidence,Lift
39520,"22916,22917,22918,22919",22921,0.023963,0.920354,34.139626
43766,"22917,22916,22918,22920,22919",22921,0.022350,0.915094,33.944525
39525,"22916,22917,22918,22920",22921,0.023502,0.910714,33.782051
29704,"22916,22917,22918",22921,0.025115,0.908333,33.693732
29724,"22916,22918,22919",22921,0.024424,0.905983,33.606545
39535,"22916,22918,22920,22919",22921,0.022811,0.900000,33.384615
29740,"22918,22917,22919",22921,0.023963,0.896552,33.256705
29728,"22916,22918,22920",22921,0.023963,0.896552,33.256705
18241,"22916,22918",22921,0.025576,0.895161,33.205128
39540,"22918,22917,22920,22919",22921,0.022350,0.889908,33.010272


In [12]:
interested.shape[0]

37

In [13]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.sort_values(by=['InvoiceDate'], inplace=True, ascending=True)
df['InvoiceDate'].head(10)

0    2010-01-12 08:26:00
1    2010-01-12 08:26:00
2    2010-01-12 08:26:00
3    2010-01-12 08:26:00
4    2010-01-12 08:26:00
5    2010-01-12 08:26:00
6    2010-01-12 08:26:00
7    2010-01-12 08:28:00
8    2010-01-12 08:28:00
24   2010-01-12 08:34:00
Name: InvoiceDate, dtype: datetime64[ns]

In [14]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    #subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 'seq_rule_input.txt', 'seq_rule_output.txt', '10%', '10%'], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [15]:
transactions = df.groupby(['InvoiceNo'])['StockCode'].apply(list)
sequences = transactions.values.tolist()

# show the first 5 sequences
sequences[:5]


[['85123', '71053', '84406', '84029', '84029', '22752', '21730'],
 ['22633', '22632'],
 ['48187',
  '21777',
  '21755',
  '21754',
  '22623',
  '84969',
  '22310',
  '22749',
  '22748',
  '22745',
  '84879',
  '22622'],
 ['22914', '22912', '22913', '22960'],
 ['21756']]

In [17]:
get_association_rules(sequences, 0.02, 0.02)

AttributeError: 'NoneType' object has no attribute 'groups'

In [19]:
#just run the java package from cmd, and read the output file.
output_dict = defaultdict(str)
outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
output_rules = []
for rule in outputs:
    left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
    sup = int(sup) / len(sequences)
    conf = float(conf)
    output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

,Left_rule,Right_rule,Support,Confidence
0,[],[],0.024248,0.439245
1,[],[],0.026559,0.220716
2,[],[],0.024565,0.204143
3,[],[],0.022707,0.188701


#### 1. Can you identify data quality issues in this dataset for performing association analysis?  
The data quality is relatively good. There is no missing value, no invalid values.
However, sevelral Variables have been dropped because they are not involved in association mining.
Some product has sub-StockCode, like 85049A belongs to 85049. They are integrated before mining.
  


#### 2. What variables did you include in the analysis and what were their roles and measurement level set? Justify your choice.  
CustomerID and StockCode are included in the analysis. We use them to discover customers' purchase behaviour. It can indicate when a customer buy one or several certain products, and then they are very likely to buy another product. We want to find out this association.

#### 3. Conduct association mining and answer the following:  
a. What is the highest lift value for the resulting rules? Which rule has this value?    
The higheset lift is 40.678733, the rule is: if a customer has bought products in ID of 21989,21987, they are very likely to buy products in ID of 21988.
b. What is the highest confidence value for the resulting rules? Which rule has this value?    
the highest confidence value is 1.0, the rule is: if a customer has bought products in ID of 23174,22698,23173,23171,22699, they are very likely to buy products in ID of 22697.
c. Discuss and explain the results. Interpret them to discuss the rule-set obtained.   
If a customer/retailer has bought PACK OF 20 SKULL PAPER NAPKINS and PACK OF 6 SKULL PAPER CUPS, we can assume they is preparing Halloween, which means PACK OF 6 SKULL PAPER PLATES would be a resonable choice for them.
Customers who bought REGENCY SUGAR BOWL GREEN, PINK REGENCY TEACUP AND SAUCER, REGENCY TEAPOT ROSES, REGENCY TEA PLATE GREEN, ROSES REGENCY TEACUP AND SAUCER, they will definitely buy GREEN REGENCY TEACUP AND SAUCER. As many customers of the company are wholesalers, they might run a tableware store so buy a similar products with different colours. 

#### 4. The store is particularly interested in products that individuals purchase when they buy “HERB MARKER CHIVES”.  
a. How many rules are in the subset?
This subset has 37 rules.
b. Based on the rules, what are the other products these individuals are most likely to purchase?  
These individuals are most likely to purchase the product of 22918,22917,22916,22919, which are HERB MARKER PARSLEY, HERB MARKER ROSEMARY,HERB MARKER THYME, HERB MARKER MINT.
We can see the products associated are all herb makers. We stop discovering more as the minimum support is only 0.02 now. 

#### 5. Can you perform sequence analysis on this dataset? If yes, present your results. If not, rationalise why?  
Yes, 4 rules have been generated. However, unfortunatelly, I can not present my results as the items on left_side and right_side are not showing correctly. 

#### 6. How the outcome of this study can be used by decision makers?  
It enables decision makers to promote the products to potential customers in a precise way. Just give them the products they are very likely to buy. It helps the efficiency of marketing.